# Punch-through listing
Scan the simulated events for start counter hits that come from tracks that passed through the lead wall at -400cm outside the radius of the beam pipe. These should be neutrons and klongs, with a few charged pions at larger radius.

In [1]:
import os
import numpy as np
os.chdir("/srv/jupyter/hdgeant4 studies")
#%pip install --user "gluex.hddm_s>=2.3.3"
from gluex import hddm_s
from gluex import xrootd_client as xclient
#%pip install --user "gluex.jupyroot>=1.0.10"
from gluex.jupyroot.hddmview import hddmview
import ROOT
#%jsroot on

In [2]:
xrdurl = "root://cn445.storrs.hpc.uconn.edu"
xrdpath = "/Gluex/resilient/simulation/KLFbeam-8-2024/"
infile = xrdurl + xrdpath + "KLFbeam11_{0}.hddm"

In [ ]:
f = ROOT.TFile("Klong_hits11.root")
tschit = f.Get("tschit")
run = 0
for row in tschit:
    if row.runNo != run:
        run = row.runNo
        hfile = hddm_s.istream(infile.format(run - 120000))
    for rec in hfile:
        breaker = False
        for pev in rec.getPhysicsEvents():
            if pev.eventNo == row.eventNo:
                for tp in rec.getStcTruthPoints():
                    x1,y1,z1 = (tp.r * np.cos(tp.phi), tp.r * np.sin(tp.phi), tp.z)
                    for to in tp.getTrackOrigins():
                        x0,y0,z0 = (to.x,to.y,to.z)
                        zint = -400
                        if z0 < zint:
                            x = x0 + (x1 - x0) * (zint - z0) / (z1 - z0)
                            y = y0 + (y1 - y0) * (zint - z0) / (z1 - z0)
                            if x**2 + y**2 > 50:
                                print("found run", run, "event", pev.eventNo)
                                print(tp)
                            break
                        x1,y1,z1 = (x0,y0,z0)
                    breaker = True
        if breaker:
            break

found run 120065 event 460999
stcTruthPoint E=0.00018719 dEdx=0.00343705 phi=1.3083 primary=true ptype=1 px=-6.92051e-05 py=-0.000142245 pz=0.000100086 r=8.19087 sector=8 t=1.40438 track=6632 z=49.2125
  trackID itrack=-2
  trackOrigin itrack=66320250 ptype=1 t=4.05959 x=-1.63317 y=64.5159 z=94.7035
  trackOrigin itrack=66320249 ptype=1 t=3.90077 x=1.63297 y=64.2241 z=98.1556
  trackOrigin itrack=66320248 ptype=1 t=3.89133 x=1.4334 y=64.2977 z=97.9689
  trackOrigin itrack=66320247 ptype=1 t=3.88048 x=1.4364 y=64.0517 z=97.7562
  trackOrigin itrack=66320117 ptype=1 t=1.4041 x=2.1286 y=7.91669 z=49.208
  trackOrigin itrack=6532 ptype=1 t=-0.559227 x=23.8401 y=52.647 z=17.7517
  trackOrigin itrack=6531 ptype=3 t=-0.632911 x=22.774 y=54.4585 z=17.0724
  trackOrigin itrack=5867 ptype=1 t=-1.49904 x=10.2415 y=75.6427 z=8.80217
  trackOrigin itrack=2687 ptype=7 t=-1.49904 x=10.2415 y=75.6427 z=8.80217
  trackOrigin itrack=2 ptype=9 t=-36.0168 x=3.24868 y=1.3584 z=-1018.83
  trackOrigin itrack